<br>

# **ATSC-405 - NUMERICAL METHODS IN METEOROLOGY | LAB 4 | WRF**

--------

## **Objectives**: Learn to run the WRF model, load data from different netcdf files, and examine WRF output in Python.

<br>

The goal of the *WRF portion* of this lab assignment is for you to become familiar with running the Weather Research and Forecasting (WRF) model on the NCAR Derecho Supercomputer along with analyzing resultant model output in Python. You will be comparing and contrasting the WRF model simulation of the 12-13 April 2022 Upper Midwest/Northern Great Plains blizzard and severe weather event that we explored in Lab 2 with a sensitivity WRF model simulation of the same event, but with a different physical-process parameterization scheme of your choosing.

<br>


- #### **PART 1**: Analyze model reflectivity output & wind at a single time
- #### **PART 2**: Analyze and compare WRF output vertical profiles
- #### **PART 3**: Analyze and compare total rainfall accumulation
- #### **PART 4**: Analyze and compare total snowfall accumulation
- #### **PART 5**: Analyze model reflectivity that may depict lateral boundary conditions or artifacts


<br>

Carefully read through and run each cell below. Some cells will require editing to successfully run and complete the lab.

<br>

*(C) Jordan Christian (jordan.christian@und.edu), Kyle Gillett (kyle.gillett@und.edu) 2024*

-----
<br>
<br>
<br>
<br>
<br>

## **IMPORT SOFTWARE PACKAGES**

Run this cell to import Python code packages that will help us load some data, process the data, conduct math operations, and build some maps.

In [ ]:
!pip install sounderpy

import sounderpy as spy             # Sounding data retrieval and analysis tool for Python
from sounderpy_wrf_module import make_wrf_profile # WRF output analysis module for SounderPy
import numpy as np                  # simple python math operations
import cartopy.crs as ccrs          # cartopy coordinate refrence system, from cartopy, for mapping -- to turn a matplotlib axis into a geoaxis (map)
import cartopy.feature as cfeature  # cartopy features, from cartopy, for mapping -- add map components to a map (borders, states, oceans, rivers, etc)
import matplotlib.pyplot as plt     # matplotlib for plotting figures, axes, graphs, etc
import metpy.calc as mpcalc         # metpy.calc provides a number of simple meteorological equations for computing variables
import scipy.ndimage as ndimage     # scipy ndimage data filtering/smoothing -- make the data 'prettier'
import netCDF4                      # netCDF4 allows us to neatly access and store .netcdf file data

-----
<br>
<br>
<br>
<br>
<br>

## **LOADING WRF OUTPUT WITH NETCDF4**

We've done this many times before -- load WRF output .nc (netCDF) files from our directory into a netcdf4 dataset so we can access and process the data with Python...

  - declaring the filename -- ex: ` file = "wrfout.nc"`
  - opening the filename with netCDF4 -- ex: ` dataset = netCDF4.Dataset(file)`

  <br>

In [ ]:
control_run_file  = 'wrfout_d01_2022-04-13_00_00_00_control_run'
sens_run_file = ?

control_run_ds = netCDF4.Dataset(?)
sens_run_ds = ?

-----
<br>
<br>
<br>
<br>
<br>

## **PART 1 | Lowest model reflectivity, 10m wind, and sounding locations**

Using the code we've used to build maps in previous labs, create a simple side-by-side figure of lowest model relfectivity and 10m horizontal winds for both simulations. On each map, plot a point of your choice of where you will plot a sounding in part 2.

+ **HINTS**
  - Refer to lab 2 and lab 3 for investigating the dataset and variables|
  - Refer to lab 2 and lab 3 for plotting data on a map.
  - To plot a point, use... `.plot(lon, lat, 'kP', markersize=20, transform=ccrs.PlateCarree(), zorder=12)`


+ **RECALL**
  - Plot reflectivity with `.pcolormesh()`
  - Plot wind barbs with `.barbs()`
  - Create a title for each subplot with `.set_title()`

+ **STEPS** *(the code block below should be structured like this)*
  1. Define variables needed for the plot...
    - the data to plot
    - the x coordinate (longitude)
    - the y coordinate (latitude)
  2. Create a colormap for reflectivity
  3. Use the `build_figure()` function
  4. Plot everything for the first subplot (control)
  5. Plot everything for the second subplot (sensitivity)
  - **ALSO** - don't forget to plot the sounding locations!

<br>

Run the cell below to create the `build_figure()` function we'll need to create side-by-side map figures


In [ ]:
# define the `build_map()` function. Changable-variables are `extent` (lat/lon bounds)
def build_figure(extent=[-105, -91, 36, 52]):

    # create figure and geoaxis | a 'geoaxis' is a normal matplotlib axis object, but providing the `projection` kwarg makes it a `geoaxis`!
    # the 'projection' is simply a mapping projection (LamertConformal, PlatteCarree, etc)
    fig, axs = plt.subplots(1, 2, figsize=(22, 10), subplot_kw={'projection': ccrs.LambertConformal(central_longitude=-98.0)})

    # flatten the axes array for easy 'looping' below
    axs = axs.flatten()

    # loop over each axis
    for ax in axs:
      # apply the map extent (lat/lon bounding box)
      ax.set_extent(extent)
      # axis aspect ratio
      ax.set_box_aspect(1)
      # state boarders
      ax.add_feature(cfeature.STATES, edgecolor='black', alpha=1, linestyle='-', linewidth=1, zorder=10)
      # land color fill
      ax.add_feature(cfeature.LAND, facecolor='gray', alpha=0.5, zorder=1)
      # ocean color fill
      ax.add_feature(cfeature.OCEAN, facecolor='gray', alpha=0.8, zorder=0)
      # country borders
      ax.add_feature(cfeature.BORDERS, color='black', alpha=1, linestyle='-', linewidth=2, zorder=11)

    # apply tight layout to the figure (keeps things tiddy)
    plt.tight_layout()

    # return the figure axis
    return fig, axs

<br>
<br>

##### **Build your figure below**

<br>

In [ ]:
###################################################################################
# STEP 1 DECLARE THE VARIABLES WE NEED

# ---- CONTROL SIMULATION ---- #
# create lat (XLAT), lon (XLONG), reflectivity (REFL_10CM),
# 10 meter u wind (U10) & 10 meter v wind (V10) variables
control_lats = control_run_ds[?][0,:,:]
control_lons = control_run_ds[?][0,:,:]
control_reflect = control_run_ds[?][0,0,:,:]
control_uwind = control_run_ds[?][0,:,:]
control_vwind = control_run_ds[?][0,:,:]

# Set reflectivity values below 10 to a NaN value (Not a Number) to clean up figures
control_reflect[control_reflect < 10] = np.NAN


# ---- SENSITIVITY RUN SIMULATION ---- #
# create lat (XLAT), lon (XLONG), reflectivity (REFL_10CM),
# 10 meter u wind (U10) & 10 meter v wind (V10) variables
sens_lats = ?
sens_lons = ?
sens_reflect = ?
sens_uwind = ?
sens_vwind = ?

# Set reflectivity values below 10 to a NaN value (Not a Number) to clean up figures
sensitive_reflect[sensitive_reflect < 10] = np.NAN
###################################################################################


###################################################################################
# STEP 2 | CREATE REFLECTIVITY COLORMAP

# create colormap for radar
# this colormap was made in Lab 3 and Lab 4 part 1
# use code from Lab 3 or Lab 4 part 1 to set your radar colormap (radar_cmap)
from matplotlib.colors import LinearSegmentedColormap
radar_cmap = ?
###################################################################################


###################################################################################
# STEP 3 | USE BUILD MAP FUNCTION
fig, axs = build_figure()
###################################################################################


###################################################################################
# STEP 4 | PLOT DATA FOR SUBPLOT 1

# ---- CONTROL SIMULATION ---- #
# plot radar pcolormesh
pmesh1 = axs[0].pcolormesh(?, ?, ?, vmin=10, vmax=75,
                          cmap=radar_cmap, alpha=1,
                           zorder=4, transform=ccrs.PlateCarree())
cbar = plt.colorbar(pmesh1, aspect=50, fraction=0.02, ax=axs[0], orientation='vertical', pad=0.02)


# plot 10m wind barbs
every = 20
axs[0].barbs(control_lons[0::every, 0::every], control_lats[0::every, 0::every],
                control_uwind[0::every, 0::every], control_vwind[0::every, 0::every],
                length=5.5, alpha=0.6, transform=ccrs.PlateCarree(), zorder=11)

# plot a sounding marker ax.plot(lon,lat,'kP',...)
# plot a '+' symbol at -94 degrees longitude and 41 degrees latitude
axs[0].plot(?, ?, 'kP', markersize=15, transform=ccrs.PlateCarree(), zorder=12)

# add a plot title
axs[0].set_title(?, fontsize=12, weight='bold')
###################################################################################


###################################################################################
# STEP 5 | PLOT DATA FOR SUBPLOT 2

# ---- SENSITIVITY RUN SIMULATION ---- #
# plot radar pcolormesh

# plot 10m wind barbs

# plot a '+' symbol at -94 degrees longitude and 41 degrees latitude

# add a plot title

###################################################################################


-----
<br>
<br>
<br>
<br>
<br>

## **PART 2 | Plotting and comparing model soundings**

##### **PLOTTING SOUNDINGS IN PYTHON**

- There are many ways this can be done. After all, it is just a x-y chart. You could make this by hand, "skewing" the temperature axis and "log-ing" the pressure axis.

- MetPy also has basic skew-T and hodograph axis objects for easy skew-t plotting.

- Or, for a full sounding plot including both a Skew-T and hodograph (plus lots more), we can use **SounderPy**. Sounderpy is a Python library specifically designed to process and plot sounding data. Learn more about SounderPy here: https://kylejgillett.github.io/sounderpy/

<br>


##### **PLOTTING A BASIC SOUNDERPY SOUNDING**

- Creating a Sounding with SounderPy requires two steps...
  1. Loading or manually creating a "clean_data" dictionary of vertical data such as pressure, temperature, height, u & v, etc
    - Here we have to manually create this dictionary. This process is a bit complicated and beyond the scope of this class, so its completed for you in the `sounderpy_wrf_module.py` file. We encourage you to peak at it, but you don't need to do anything to it. Just run the `make_wrf_profile` function.
  2. Passing that dictionary into SounderPy's `.build_sounding()` function.

<br>

Lets make a basic sounding below. To create a `clean_data` dictionary, use...
  - `make_wrf_profile(dataset, [latitude, longitude])`
    - where "dataset" is your control or sensitivity dataset name and "latitude" and "longitude" is lat/lon point of your sounding (in [ ]!)

<br>

In [ ]:
# Build a SounderPy 'clean_data' dictionary of WRF output to plot a sounding.
# Plot sounding location at 41 degrees latitude and -94 degrees longitude
clean_data = make_wrf_profile(control_run_ds, [?, ?])

# Now, pass the 'clean_data' dictionary into the SounderPy `build_sounding` function
spy.build_sounding(clean_data, special_parcels='simple')

<br>
<br>


##### **PLOTTING A SOUNDERPY COMPARISON SOUNDING**

- Now we can create a sounding figure with multiple profiles using Sounderpy's `build_composite()` function, specifically designed to plot multiple profiles at once for comparison.

<br>

To do so...
  1. Create a 'clean_data' dictionary of data for both the control run and sensitivity run
  2. Add those variables to a list and pass it into the `build_composite()` function

<br>

In [ ]:
# create dictionaries of data
# plot sounding location at 41 degrees latitude and -94 degrees longitude
control_run_profile   = make_wrf_profile(control_run_ds, [?, ?], run_name='?')
sens_run_profile = make_wrf_profile(?)

# add them to a list
data_list = [control_run_profile, sens_run_profile]

# build the sounding with a few extra settings to make a nice plot
spy.build_composite(data_list, colors_to_use=['firebrick', 'cyan'], lw_to_use=[4,4], dark_mode=True)

-----
<br>
<br>
<br>
<br>
<br>

## **PART 3 | Comparing rainfall accumulation output**

Using any code from previous labs and our maps above, create a side-by-side map figure of rainfall accumulation for both runs.

Hint: use the `build_figure()` function again

<br>

In [ ]:
###################################################################################
# STEP 1 DECLARE THE VARIABLES WE NEED

# ---- CONTROL SIMULATION ---- #
control_lats = ?
control_lons = ?
# Add RAINNC with RAINC to get total precipitation
control_total_precip = control_run_ds[?][0,:,:] + control_run_ds[?][0,:,:]

# ---- SENSITIVITY SIMULATION ---- #
sens_lats = ?
sens_lons = ?
sens_total_precip = ?
###################################################################################


###################################################################################
# CREATE A RAINFALL COLORMAP
from matplotlib.colors import LinearSegmentedColormap
precip_cmap  = LinearSegmentedColormap.from_list('custom_cmap',['#FFFFFF','#2FFFB8','#7FFF00','#00CD00','#008B00','#104E8B','#1E90FF',
                                                              '#00B2EE','#00EEEE','#8968CD','#912CEE','#8B008B', '#8B0000',
                                                              '#CD0000','#EE4000', '#FF7F00', '#CD8500', '#FFD700', '#FFFF00',
                                                              '#FFAEB9'],N=256)
###################################################################################


###################################################################################
# USE BUILD FIGURE FUNCTION
fig, axs = ?
###################################################################################


###################################################################################
# PLOT DATA FOR SUBPLOT 1
# ---- CONTROL SIMULATION ---- #
# plot rainfall pcolormesh
pmesh1 = axs[0].pcolormesh(?, ?, ?, vmin=0, vmax=205, cmap=precip_cmap,
                           alpha=1, zorder=4, transform=ccrs.PlateCarree())
cbar = plt.colorbar(pmesh1, aspect=50, fraction=0.02, ax=axs[0], orientation='vertical', pad=0.02)

# add a plot title
axs[0].set_title('?', fontsize=12, weight='bold')
###################################################################################


###################################################################################
# PLOT DATA FOR SUBPLOT 2
# ---- SENSITIVITY SIMULATION ---- #
# plot rainfall pcolormesh

# add a plot title

###################################################################################

-----
<br>
<br>
<br>
<br>
<br>

## **PART 4 | Comparing snowfall accumulation output**

Using any code from previous labs and our maps above, create a side-by-side map figure of snowfall accumulation for both runs.

Hint: use the `build_figure()` function again

<br>

In [ ]:
###################################################################################
# Make this figure completely on your own! Use previous code to help write
# each code section

# STEP 1 DECLARE THE VARIABLES WE NEED

# ---- CONTROL SIMULATION ---- #
# SNOWH variable provides snow depth
?

# ---- SENSITIVITY SIMULATION ---- #
# SNOWH variable provides snow depth
?

###################################################################################


###################################################################################
# CREATE A RAINFALL COLORMAP
?
snow_cmap  = LinearSegmentedColormap.from_list('custom_cmap',['#FFFFFF','#bed7e7','#69aed5','#2c82bd','#024f9c','#022195',
                                                              '#fefe96','#fec400','#fe8600'],N=256)
###################################################################################


###################################################################################
# USE BUILD MAP FUNCTION
?

###################################################################################


###################################################################################
# PLOT DATA FOR SUBPLOT 1
# ---- CONTROL SIMULATION ---- #
?

###################################################################################


###################################################################################
# PLOT DATA FOR SUBPLOT 2
# ---- SENSITIVITY SIMULATION ---- #
?

###################################################################################